# Multiple Linear Regression with sklearn - Exercise Solution

You are given a real estate dataset. 

Real estate is one of those examples that every regression course goes through as it is extremely easy to understand and there is a (almost always) certain causal relationship to be found.

The data is located in the file: 'real_estate_price_size_year.csv'. 

You are expected to create a multiple linear regression (similar to the one in the lecture), using the new data. 

Apart from that, please:
-  Display the intercept and coefficient(s)
-  Find the R-squared and Adjusted R-squared
-  Compare the R-squared and the Adjusted R-squared
-  Compare the R-squared of this regression and the simple linear regression where only 'size' was used
-  Using the model make a prediction about an apartment with size 750 sq.ft. from 2009
-  Find the univariate (or multivariate if you wish - see the article) p-values of the two variables. What can you say about them?
-  Create a summary table with your findings

In this exercise, the dependent variable is 'price', while the independent variables are 'size' and 'year'.

Good luck!

## Import the relevant libraries

In [10]:
# Import the relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import os
import seaborn as sns
# We can override the default matplotlib styles with those of Seaborn
sns.set()

## Load the data

In [2]:
df = pd.read_csv(os.path.join('real_estate_price_size_year.csv'))

In [4]:
df.head()

,price,size,year
0,234314.144,643.09,2015
1,228581.528,656.22,2009
2,281626.336,487.29,2018
3,401255.608,1504.75,2015
4,458674.256,1275.46,2009


## Create the regression

### Declare the dependent and the independent variables

In [5]:
# There are two independent variables: 'Size' and 'Year'
x = df[['size','year']]

# and a single depended variable: 'Price'
y = df['price']

### Regression

In [47]:
# Since the p-values are obtained through certain statistics, we need the 'stat' module from scipy.stats
import scipy.stats as stat

# Since we are using an object oriented language such as Python, we can simply define our own 
# LinearRegression class (the same one from sklearn)
# By typing the code below we will ovewrite a part of the class with one that includes p-values
# Here's the full source code of the ORIGINAL class: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/linear_model/base.py#L362


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    # nothing changes in __init__
    # def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
        #          n_jobs=1):
        # self.fit_intercept = fit_intercept
        # self.normalize = normalize
        # self.copy_X = copy_X
        # self.n_jobs = n_jobs

    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        
        # Calculate SSE (sum of squared errors)
        # and SE (standard error)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # compute the t-statistic for each feature
        self.t = self.coef_ / se
        # find the p-value for each feature
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        return self

In [42]:
# We start by creating a linear regression object
reg = LinearRegression()

# The whole learning process boils down to fitting the regression
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

### Find the intercept

In [43]:
reg.intercept_

-5772267.017463278

### Find the coefficients

In [44]:
reg.coef_

array([ 227.70085401, 2916.78532684])

### Calculate the R-squared

In [45]:
reg.score(x,y)

0.7764803683276793

### Calculate the Adjusted R-squared

In [19]:
def get_adj_r_sq(feature, target):
    r2 = reg.score(feature,target)
    n = feature.shape[0]
    p = feature.shape[1]

    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [20]:
get_adj_r_sq(x,y)

0.77187171612825

### Compare the R-squared and the Adjusted R-squared

It seems the the R-squared is only slightly larger than the Adjusted R-squared, implying that we were not penalized a lot for the inclusion of 2 independent variables.

### Compare the Adjusted R-squared with the R-squared of the simple linear regression

Comparing the Adjusted R-squared with the R-squared of the simple linear regression (when only 'size' was used), we realize that 'Year' is not bringing too much value to the result.

### Making predictions

Find the predicted price of an apartment that has a size of 750 sq.ft. from 2009.

In [24]:
# new_df = pd.DataFrame({'size': [750], 'year':[2009]})

In [28]:
reg.predict([[750, 2009]])

array([258330.34465995])

In [46]:
reg.p

array([8.12763222e-31, 3.57340758e-01])

### Calculate the univariate p-values of the variables

In [30]:
from sklearn.feature_selection import f_regression

In [31]:
f_regression(x,y)

(array([285.92105192,   0.85525799]), array([8.12763222e-31, 3.57340758e-01]))

In [32]:
p_values = f_regression(x,y)[1]
p_values

array([8.12763222e-31, 3.57340758e-01])

In [33]:
p_values.round(3)

array([0.   , 0.357])

### Create a summary table with your findings

In [49]:
# Let's create a new data frame with the names of the features
reg_summary = pd.DataFrame([['size'],['year']],columns =['Features'])
# Then we create and fill a second column, called 'Coefficients' with the coefficients of the regression
reg_summary['Coefficients'] = reg.coef_
# Finally, we add the p-values we just calculated
reg_summary['p-values'] = p_values.round(3)

In [40]:
# This result is identical to the one from StatsModels
reg_summary

,Features,Coefficients,p-values
0,size,227.700854,0.000
1,year,2916.785327,0.357


It seems that 'Year' is not event significant, therefore we should remove it from the model.